## Scrapping one page (Immovlan)

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd

In [ ]:
# Url of website
url = 'https://immovlan.be/en/detail/residence/for-sale/2500/lier/rbt69212'


In [ ]:
ua = UserAgent()

In [ ]:
headers =  {

        "User-Agent": ua.random,
       
        'Accept': (
            'text/html,application/xhtml+xml,application/xml;q=0.9,'
            'image/avif,image/webp,*/*;q=0.8'
        ),
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'TE': 'trailers',
    }


In [ ]:
s = requests.Session()
s.headers.update(headers)

In [ ]:
response = s.get(url)
content =response.content
print(response)

In [ ]:
soup = BeautifulSoup(content, 'html.parser')
soup

##### Get characteristics for property

In [ ]:
property_characteristics = dict()

In [ ]:
code = soup.find(class_='vlancode')
property_characteristics['property_code'] = code.text

In [ ]:
type_of_property = soup.find(class_= 'detail__header_title_main')
property_characteristics['type_of_property'] = type_of_property.text.split()[0]

In [ ]:
price = soup.find(class_='detail__header_price_data')
property_characteristics['price'] = price.text

In [ ]:
locality = soup.find(class_='city-line')
property_characteristics['locality'] = locality.text

In [ ]:
for tag in soup.find_all('h4', class_ =False):
    characteristic_name = "_".join(list(map(str.lower, tag.text.split())))
    property_characteristics[characteristic_name] = tag.find_next().text

In [ ]:
for k,v in property_characteristics.items():
    print(f"{k}: {v}")

#### pandas to create a file with all properties

In [ ]:
properties = pd.DataFrame.from_records([property_characteristics])
properties

## Scrapping a list of properties

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
from time import time, sleep
import pandas as pd


In [ ]:
properties = []

In [ ]:
df = pd.read_csv("final_listing_links.csv", header=0, nrows=50)
urls =df['link'].tolist()
urls

In [ ]:
## Function definition to obtain individual properties

def get_raw_property(url: str, session:requests.Session) -> BeautifulSoup:

    start_time = time()

    sleep(random.uniform(1.5, 3))
    
    try: 
        response = session.get(url)
        content =response.content
        print(response)
        soup = BeautifulSoup(content, 'html.parser')
    except Exception as e:
        print(f"Error {e}")
    
    end_time = time()
    duration = end_time - start_time
    print(f"This scrap has taken {duration}")
    
    return soup    


In [ ]:
## Function to obtain a dictionary with the properties
def get_property_characteristics(url:str, soup:BeautifulSoup) -> dict:
    
    property_characteristics = dict()

    code = soup.find(class_='vlancode')
    property_characteristics['property_code'] = code.text

    type_of_property = soup.find(class_= 'detail__header_title_main')
    property_characteristics['type_of_property'] = type_of_property.text.split()[0]

    price = soup.find(class_='detail__header_price_data')
    property_characteristics['price'] = price.text

    locality = soup.find(class_='city-line')
    property_characteristics['locality'] = locality.text

    for tag in soup.find_all('h4', class_ =False):
        characteristic_name = "_".join(list(map(str.lower, tag.text.split())))
        property_characteristics[characteristic_name] = tag.find_next().text

    property_characteristics['property_url'] = url
        
    for k,v in property_characteristics.items():
        print(f"{k}: {v}")

    return property_characteristics

In [ ]:
# Establishing session with headers for connection
ua = UserAgent()
headers =  {

        "User-Agent": ua.random,
       
        'Accept': (
            'text/html,application/xhtml+xml,application/xml;q=0.9,'
            'image/avif,image/webp,*/*;q=0.8'
        ),
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'TE': 'trailers',
    }

s = requests.Session()
s.headers.update(headers)


In [ ]:

for url in urls:
    soup = get_raw_property(url, s)
    properties.append(get_property_characteristics(url,soup))


In [ ]:
properties


##### Create a dataframe with all properties

In [ ]:
df_properties = pd.json_normalize(properties)
df_properties

In [ ]:
df_properties.to_csv('out.csv', index=False, encoding = "utf-8")

In [ ]:
has_header = False

## Preliminar Cleaning

In [21]:
from typing import List
import pandas as pd
import os

In [22]:
def url_file_has_header(file_name) -> None:
    with open(file_name, "r", encoding="utf-8") as f:
        sample = f.readline()
        return "http" not in sample.lower()

In [23]:
def open_csv_file(file_name: str, header: bool=True) -> pd:

    #header = file_has_header(file_name)
    try:
        if header:
            df = pd.read_csv(file_name, header=0)
        else:
            df = pd.read_csv(file_name, header=None)

    except Exception as e:
        print(f"Error {e}")

    return df

In [24]:
def removing_urls(string_to_remove: str, df: pd) -> pd:
    try:
        final_df = df[~df.iloc[:, 0].str.contains(string_to_remove)]

    except Exception as e:
        print(f"Error {e}")

    return final_df


In [ ]:
preliminar_df = open_csv_file("listing_links-1-3-prov.csv")
preliminar_df

In [ ]:
fin_df= removing_urls('project', preliminar_df)
fin_df

### Cleaning columns from scrapped properties

In [19]:
raw_df = open_csv_file("data/scrapped_properties_10-11.csv")
raw_df

C:\Users\herms\AppData\Local\Temp\ipykernel_10536\586911360.py:7: DtypeWarning: Columns (5,7,12,14,15,19,20,21,31,47,59,64,75,76,77,78,99,100,101,102,103,104,105,107,109,110,111,112,113,118,119,120,121,122,123,124,126,127,128,129,130,131,132,135,136,137,138,140,141,142,143,144,145,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,32

,0,1,2,3,4,5,6,7,8,9,...,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917
0,property_code,type_of_property,price,locality,state_of_the_property,build_year,availability,number_of_bedrooms,livable_surface,furnished,...,d478e767-7c81-49ea-83ed-d1a452dc9bfc_404341984...,98f7329e-99a9-45e1-baa4-f92b24ec1b60_404341984...,6d659931-2dc9-4ef8-a637-88befdf2b2e4_404341984...,942a56d6-9202-4f61-be20-edeeb3bdd2dd_3460cahie...,5faf3502-3029-476e-bcf8-fd1330901087_3382cahie...,c5f7884c-c8a0-4eea-9a0b-2ea1ac6b642d_4001plan_...,913921b9-9599-4ce3-92c6-0323e07a4619_4000plan_...,7f4d0a89-7db8-43ba-9f0a-b4f22cc9822e_3999plan_...,652b3ad3-8050-428c-9af3-3bb664f25c00.pdf,f1d57786-9547-4b5f-b3f6-7e0708d7c27a.pdf
1,VBC89368,Residence,240 000 €,5575 Willerzie,To be renovated,1979,On contract,5,144 m²,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VBC89366,Residence,195 000 €,5300 Andenne,To be renovated,1950,On contract,3,140 m²,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VBC89196,Residence,335 500 €,5190 Moustier-Sur-Sambre,NaN,NaN,NaN,3,212 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VBC88779,Villa,345 000 €,5590 Ciney,To be renovated,1970,On contract,4,180 m²,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,VBA82854,Apartment,Price on request,6900 Marche-en-Famenne,NaN,NaN,NaN,2.0,105 m²,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1886,VBA53602,Mixed,388 000 €,6980 La Roche-en-Ardenne,Normal,1850.0,NaN,4.0,380 m²,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1887,VBA51895,Apartment,120 000 €,6800 Libramont-Chevigny,NaN,NaN,NaN,2.0,50 m²,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1888,VBA38853,Residence,550 000 €,6900 Aye,Normal,1968.0,On contract,2.0,216 m²,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Download,NaN


In [6]:
def columns_selection(df: pd) -> List[str]:
    columns_to_eliminate = []
    all_columns = list(df.columns.values)

    columns_to_eliminate =[column for column in all_columns if column.endswith(('.pdf', '.png', '.jpg', 'jpeg', 'docx', 'termsandconditions'))]
    
    columns_2 = [column for column in all_columns if column.startswith('documents?id')]
    columns_to_eliminate.extend(columns_2)

    columns_3 = [column for column in all_columns if  'score_represents' in column ]

    #columns_4 = [column for column in all_columns if column in ['demarcated_flooding_area', ]]

    columns_to_eliminate.extend(columns_3)

    print(len(columns_to_eliminate))
    return columns_to_eliminate

In [ ]:
#Inplace False will return other dataframe
def column_cleaning(df: pd, columns_to_clean: List[str], in_place: bool = False) -> pd:
    try:
        df= df.drop(columns = columns_to_clean, inplace = in_place)
        
    except Exception as e:
        print(f"Error {e}")
    
    #dropping columns that have less than 10% values
    clean_pd = df.dropna(axis=1, thresh=0.10*len(df))
    return clean_pd

In [ ]:
cols_to_eli= columns_selection(raw_df)


In [ ]:
clean_pd = column_cleaning(raw_df, columns_selection(raw_df))
clean_pd

##### Creating dataframes for each file

In [28]:
properties_file_names = ["scrapped_properties_1-3.csv", "scrapped_properties_4-6.csv", "scrapped_properties_7-9.csv",
                         "scrapped_properties_10-11.csv"]

In [ ]:
def merge_properties(file_names) -> pd:
    all_dfs = []
    for file in file_names:
        file_path = os.path.join("data", file)
        raw_df = open_csv_file(file_path)
        clean_df= column_cleaning(raw_df, columns_selection(raw_df))
        clean_df = 
        all_dfs.append(clean_df)

    total_df = pd.concat(all_dfs, ignore_index=True, sort = True)
    total_df

    return total_df

In [31]:
new_df = merge_properties(properties_file_names)

C:\Users\herms\AppData\Local\Temp\ipykernel_10536\1038028522.py:6: DtypeWarning: Columns (31,54,59,60,61,62,63,74,79,80,81,82,83,85,87,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,109,110,111,112,114,115,116,117,118,119,120,121,123,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,166,167,168,169,170,171,172,173,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,3

1936


C:\Users\herms\AppData\Local\Temp\ipykernel_10536\1038028522.py:6: DtypeWarning: Columns (53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,74,77,92,94,96,101,105,106,107,108,109,110,112,113,114,115,116,117,118,119,120,121,122,123,127,128,129,130,131,132,133,134,135,136,137,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,163,164,165,166,167,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,190,191,192,193,194,195,196,197,198,199,202,203,204,205,206,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,33

1589


C:\Users\herms\AppData\Local\Temp\ipykernel_10536\1038028522.py:6: DtypeWarning: Columns (52,61,62,77,81,82,83,84,85,87,88,89,94,96,100,101,102,104,107,108,109,110,111,112,113,114,115,116,118,119,120,121,122,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,149,150,151,152,153,155,157,158,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,

1646


C:\Users\herms\AppData\Local\Temp\ipykernel_10536\1038028522.py:6: DtypeWarning: Columns (64,76,77,78,99,100,101,102,103,104,105,107,110,111,112,113,118,119,120,121,122,123,124,126,127,128,129,130,131,132,135,136,137,138,140,141,142,143,144,145,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337

1793


In [33]:
new_df

,access_for_disabled,air_conditioning,alarm,attic,availability,balcony,bike_storage,build_year,buildable_surface,building_permission_granted,...,the_property_and/or_its_surroundings_are_protected.,total_land_surface,type_of_glazing,type_of_heating,type_of_property,urbanism_affectation,validity_date_epc/peb,veranda,wash_room,yearly_total_primary_energy_consumption
0,NaN,NaN,NaN,Yes,NaN,NaN,NaN,1899.0,61 m²,No,...,NaN,81 m²,NaN,Gas,Residence,"Residential zone (residential, urban)",NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,No,On contract,NaN,No,1980.0,NaN,NaN,...,NaN,505 m²,NaN,Fuel oil,Residence,"Residential zone (residential, urban)",NaN,No,No,NaN
2,NaN,NaN,NaN,No,Immediately,NaN,No,1967.0,NaN,NaN,...,NaN,NaN,NaN,NaN,Apartment,"Residential zone (residential, urban)",03/10/2024,No,No,NaN
3,NaN,NaN,NaN,No,Immediately,NaN,No,2023.0,NaN,NaN,...,NaN,NaN,NaN,Gas,Apartment,"Residential zone (residential, urban)",NaN,No,No,NaN
4,NaN,NaN,NaN,No,Immediately,NaN,No,2023.0,NaN,NaN,...,NaN,NaN,NaN,Gas,Apartment,"Residential zone (residential, urban)",NaN,No,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Double glass,Gas,Apartment,NaN,NaN,NaN,NaN,NaN
9968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1850.0,NaN,NaN,...,NaN,463 m²,Double glass,Fuel oil,Mixed,NaN,24/11/2033,NaN,NaN,16400 kWh/year
9969,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Double glass,Fuel oil,Apartment,NaN,02/06/2033,NaN,NaN,14887 kWh/year
9970,Yes,NaN,NaN,NaN,On contract,NaN,NaN,1968.0,NaN,NaN,...,NaN,1207 m²,Double glass,NaN,Residence,NaN,NaN,NaN,NaN,NaN


In [32]:
new_df.columns.values

array(['access_for_disabled', 'air_conditioning', 'alarm', 'attic',
       'availability', 'balcony', 'bike_storage', 'build_year',
       'buildable_surface', 'building_permission_granted', 'cellar',
       'certification_-_electrical_installation', 'co2_emission',
       'currently_leased', 'demarcated_flooding_area', 'diningroom',
       'dressing', 'elevator', 'entry_phone', 'epc/peb_reference',
       'flooding_area_type', 'floor_of_appartment', 'frontage_width',
       'furnished', 'garage', 'garden', 'garden_orientation', 'gas',
       'ground_depth', 'heat_pump', 'kitchen_equipment', 'kitchen_type',
       'livable_surface', 'locality', 'number_of_bathrooms',
       'number_of_bedrooms', 'number_of_facades', 'number_of_floors',
       'number_of_garages', 'number_of_parking_places_(outdoor)',
       'number_of_parking_spaces_(indoor)', 'number_of_showers',
       'number_of_toilets', 'orientation_of_the_front_facade',
       'planning_permission_granted', 'preemption_right', 'p

In [34]:
columns_to_preserve = [
       'furnished', 
       'garage', 
       'garden',  
       'kitchen_equipment', 
       'kitchen_type',
       'livable_surface', 
       'locality', 
       'number_of_bathrooms',
       'number_of_bedrooms', 
       'number_of_facades', 
       'number_of_floors',
        'price',
       'property_code', 
       'property_url',  
       'state_of_the_property',
        'surface_garden', 
       'surface_terrace', 
       'swimming_pool',
       'terrace', 
       'total_land_surface', 
       'type_of_heating',
       'type_of_property', ]

In [35]:
len(columns_to_preserve)

22

In [37]:
columns_to_drop = [column for column in new_df.columns.values if column not in columns_to_preserve]
columns_to_drop

['access_for_disabled',
 'air_conditioning',
 'alarm',
 'attic',
 'availability',
 'balcony',
 'bike_storage',
 'build_year',
 'buildable_surface',
 'building_permission_granted',
 'cellar',
 'certification_-_electrical_installation',
 'co2_emission',
 'currently_leased',
 'demarcated_flooding_area',
 'diningroom',
 'dressing',
 'elevator',
 'entry_phone',
 'epc/peb_reference',
 'flooding_area_type',
 'floor_of_appartment',
 'frontage_width',
 'garden_orientation',
 'gas',
 'ground_depth',
 'heat_pump',
 'number_of_garages',
 'number_of_parking_places_(outdoor)',
 'number_of_parking_spaces_(indoor)',
 'number_of_showers',
 'number_of_toilets',
 'orientation_of_the_front_facade',
 'planning_permission_granted',
 'preemption_right',
 'running_water',
 'sewer_connection',
 'solar_panels',
 'specific_primary_energy_consumption',
 'surface_bedroom_1',
 'surface_bedroom_2',
 'surface_bedroom_3',
 'surface_bedroom_4',
 'surface_kitchen',
 'surface_of_living-room',
 'surface_of_the_bathroom(s)

In [38]:
new_clean_df= column_cleaning(new_df, columns_to_drop)
new_clean_df

,furnished,garage,garden,kitchen_equipment,kitchen_type,livable_surface,locality,number_of_bathrooms,number_of_bedrooms,number_of_facades,...,property_code,property_url,state_of_the_property,surface_garden,surface_terrace,swimming_pool,terrace,total_land_surface,type_of_heating,type_of_property
0,No,NaN,Yes,Partially equipped,NaN,99 m²,2500 Lier,1.0,3.0,2.0,...,RBT69212,https://immovlan.be/en/detail/residence/for-sa...,To be renovated,18 m²,18 m²,NaN,Yes,81 m²,Gas,Residence
1,No,Yes,Yes,Super equipped,NaN,123 m²,2440 Geel,1.0,3.0,3.0,...,RBT67563,https://immovlan.be/en/detail/residence/for-sa...,Normal,NaN,NaN,NaN,Yes,505 m²,Fuel oil,Residence
2,No,NaN,NaN,NaN,NaN,102 m²,2260 Zoerle-Parwijs,1.0,2.0,2.0,...,RBT67578,https://immovlan.be/en/detail/apartment/for-sa...,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Apartment
3,No,NaN,Yes,Super equipped,NaN,93 m²,2500 Lier,1.0,2.0,2.0,...,RBT67552,https://immovlan.be/en/detail/apartment/for-sa...,New,NaN,8 m²,NaN,Yes,NaN,Gas,Apartment
4,No,NaN,Yes,Super equipped,NaN,116 m²,2500 Lier,1.0,2.0,2.0,...,RBT67551,https://immovlan.be/en/detail/apartment/for-sa...,New,27 m²,8 m²,NaN,Yes,NaN,Gas,Apartment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,No,Yes,Yes,NaN,NaN,105 m²,6900 Marche-en-Famenne,1.0,2.0,NaN,...,VBA82854,https://immovlan.be/en/detail/apartment/for-sa...,NaN,94 m²,13 m²,NaN,Yes,NaN,Gas,Apartment
9968,No,NaN,NaN,NaN,NaN,380 m²,6980 La Roche-en-Ardenne,4.0,4.0,4.0,...,VBA53602,https://immovlan.be/en/detail/mixed-building/f...,Normal,NaN,NaN,NaN,NaN,463 m²,Fuel oil,Mixed
9969,No,NaN,No,Fully equipped,Western kitchen,50 m²,6800 Libramont-Chevigny,1.0,2.0,2.0,...,VBA51895,https://immovlan.be/en/detail/apartment/for-sa...,NaN,NaN,NaN,No,No,NaN,Fuel oil,Apartment
9970,Yes,NaN,Yes,Fully equipped,Western kitchen,216 m²,6900 Aye,1.0,2.0,4.0,...,VBA38853,https://immovlan.be/en/detail/residence/for-sa...,Normal,1041 m²,83 m²,No,Yes,1207 m²,NaN,Residence


In [49]:

valid_df = new_clean_df[~new_clean_df['property_code'].isnull()]

In [50]:
valid_df

,furnished,garage,garden,kitchen_equipment,kitchen_type,livable_surface,locality,number_of_bathrooms,number_of_bedrooms,number_of_facades,...,property_code,property_url,state_of_the_property,surface_garden,surface_terrace,swimming_pool,terrace,total_land_surface,type_of_heating,type_of_property
0,No,NaN,Yes,Partially equipped,NaN,99 m²,2500 Lier,1.0,3.0,2.0,...,RBT69212,https://immovlan.be/en/detail/residence/for-sa...,To be renovated,18 m²,18 m²,NaN,Yes,81 m²,Gas,Residence
1,No,Yes,Yes,Super equipped,NaN,123 m²,2440 Geel,1.0,3.0,3.0,...,RBT67563,https://immovlan.be/en/detail/residence/for-sa...,Normal,NaN,NaN,NaN,Yes,505 m²,Fuel oil,Residence
2,No,NaN,NaN,NaN,NaN,102 m²,2260 Zoerle-Parwijs,1.0,2.0,2.0,...,RBT67578,https://immovlan.be/en/detail/apartment/for-sa...,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Apartment
3,No,NaN,Yes,Super equipped,NaN,93 m²,2500 Lier,1.0,2.0,2.0,...,RBT67552,https://immovlan.be/en/detail/apartment/for-sa...,New,NaN,8 m²,NaN,Yes,NaN,Gas,Apartment
4,No,NaN,Yes,Super equipped,NaN,116 m²,2500 Lier,1.0,2.0,2.0,...,RBT67551,https://immovlan.be/en/detail/apartment/for-sa...,New,27 m²,8 m²,NaN,Yes,NaN,Gas,Apartment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,No,Yes,Yes,NaN,NaN,105 m²,6900 Marche-en-Famenne,1.0,2.0,NaN,...,VBA82854,https://immovlan.be/en/detail/apartment/for-sa...,NaN,94 m²,13 m²,NaN,Yes,NaN,Gas,Apartment
9968,No,NaN,NaN,NaN,NaN,380 m²,6980 La Roche-en-Ardenne,4.0,4.0,4.0,...,VBA53602,https://immovlan.be/en/detail/mixed-building/f...,Normal,NaN,NaN,NaN,NaN,463 m²,Fuel oil,Mixed
9969,No,NaN,No,Fully equipped,Western kitchen,50 m²,6800 Libramont-Chevigny,1.0,2.0,2.0,...,VBA51895,https://immovlan.be/en/detail/apartment/for-sa...,NaN,NaN,NaN,No,No,NaN,Fuel oil,Apartment
9970,Yes,NaN,Yes,Fully equipped,Western kitchen,216 m²,6900 Aye,1.0,2.0,4.0,...,VBA38853,https://immovlan.be/en/detail/residence/for-sa...,Normal,1041 m²,83 m²,No,Yes,1207 m²,NaN,Residence
